> Introdución al procesado de lenguaje natural

El Procesamiento de Lenguaje Natural (NLP, del inglés *Natural Language Processing*) sirve para enseñar a los ordenadores a entender y trabajar con el lenguaje humano (las palabras que usamos todos los días). Desde chatbots y aplicaciones de traducción hasta filtros de spam y asistentes de voz, el NLP impulsa muchas herramientas en las que confiamos sin siquiera notarlo. En este notebook, exploraremos algunas de las técnicas básicas que permiten a los ordenador manipular texto de manera eficaz.

# La tarea

Sigue el notebook tratando de *entender* lo que estás haciendo. Por el camino, encontrarás algunos <font color='red'>TO-DO</font>s que plantean preguntas relacionadas con lo que acabas de hacer o te piden que escribas algún pequeño fragmento de código (Python) (ya sea para responder la pregunta o para continuar). Este notebook es solo una herramienta para que aprendas y **no** está pensado para ser entregado (de cara a la evaluación) una vez que termines.

Fíjate que puedes pedir ayuda al *chatbot* integrado de *Colab* (*Gemini*) en cualquier momento. Está permitido, incluso recomendado (si uno es consciente de sus limitaciones y los posibles errores que puede cometer), pero ten en cuenta que en última instancia debes entender lo que está sucediendo (tener una "visión general").

---

# Curso rápido de notebooks [Jupyter](https://jupyter.org/)/[Colab](https://colab.research.google.com/?hl=en)

Un notebook *jupyter*/*colab* tiene dos tipos de celdas:

- *celdas de código*, pensadas para escribir y ejecutar código (diferentes lenguajes están soportados), y

- *celdas markdown* para texto, imágenes, títulos, etc. (*esta* celda es una celda markdown).

Casi todo se puede hacer con la interfaz (menús de arriba y botones junto a las celdas), pero usar atajos de teclado es muy práctico para ir rápido.

## **Modos de Celda**
- **Modo Edición**: Pulsa `Enter` para editar una celda.
- **Modo Comando**: Pulsa `Esc` para interactuar con el notebook.

## **Atajos de teclado (Modo Comando y Edición)**
- **Ejecutar celda**: `Shift + Enter`
- **Ejecutar celda e insertar debajo**: `Alt + Enter`

## **Atajos (Modo Comando)**
- **Insertar celda debajo**: `b`
- **Insertar celda arriba**: `a`
- **Borrar celda**: `d, d` (pulsa `d` dos veces)
- **Cambiar a celda de código**: `y`
- **Cambiar a celda markdown**: `m`

## **Atajos (Modo Edición)**
- **Autocompletar**: `Tab`

*Google Colab* tiene integrado [Gemini](https://gemini.google.com) (la herramienta tipo ChatGPT de *Google*). Usando el botón *Gemini* al lado de una celda de código puedes pedir, por ejemplo, que te explique el código.

Los notebooks tienen un estado interno (determinado por el código ya ejecutado), por lo que están **pensados para ejecutarse secuencialmente**. Normalmente empiezas por la primera celda y vas pulsando `Shift + Enter` para ejecutar la celda actual (después de leerla) y pasar a la siguiente.

<font color='orange'>ADVERTENCIA</font>: Al ejecutar la primera *celda de código* del notebook verás una advertencia que dice que el notebook no fue creado por *Google*. Haz clic en "Run anyway" (no hay peligro: el código ni siquiera se ejecuta localmente).

<font color='green'>SUGERENCIA</font>: Si seleccionas `Save a copy in Drive` en el menú `File` puedes guardar tu propia copia del notebook (con los cambios que hagas). Te pedirá que inicies sesión con tu cuenta *Google* (UC3M).

---

## Preparación del entorno

Algunos `import`s

In [ ]:
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS, TfidfTransformer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

A continuación usaremos dos recursos de [nltk](https://www.nltk.org) que no están instalados junto con la librería.

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

<font color='red'>TO-DO</font> ¿Cuáles son los dos recursos de NLTK que descargamos anteriormente?

# Preprocesamiento de Texto

El texto es inherentemente *datos no estructurados* en el sentido de que no sigue un formato fijo (no es una tabla, matriz o hoja de cálculo ordenada). Por lo tanto, se requiere cierto *preprocesamiento* antes de que el texto pueda ser alimentado a un modelo de aprendizaje automático (ML). Aprender a hacer este *preprocesamiento* es el propósito de esta sección.

Observa que
> Como los algoritmos de aprendizaje automático procesan números en lugar de texto, el texto debe convertirse a números.
>
> -- <cite>[Wikipedia](https://en.wikipedia.org/wiki/Large_language_model#Tokenization)</cite>

### Tokenización

La tokenización es el proceso de dividir un texto en trozo más pequeños, llamados tokens. Estos tokens pueden ser palabras, frases, o incluso oraciones. A cada token se le asigna entonces un número (único). La tokenización es un paso crucial en NLP ya que nos permite analizar y procesar datos textuales de manera más efectiva.

ADVERTENCIA: Esto es notablemente diferente de la [tokenización *léxica*](https://en.wikipedia.org/wiki/Lexical_analysis#Lexical_token_and_lexical_tokenization), que no tiene interés para nosotros en este curso.

Vamos a *tokenizar* el siguiente texto (puedes sustituirlo por cualquier texto que te guste)


In [ ]:
text = 'They can kill you, but the legalities of eating you are quite a bit dicier (from "Infinite Jest", by David Foster Wallace)'

utilizando la librería `nltk`

In [ ]:
tokens = word_tokenize(text)
print("Tokens:", tokens)

Con este *tokenizador* concreto cada token es una palabra, y los signos de puntuación se tratan como tokens separados. Esto es útil para muchas tareas de NLP, pero hay otros tokenizadores que pueden manejar la puntuación de manera diferente o tokenizar a nivel de carácter.

Después de la tokenización, simplemente numeras los tokens para que cada uno tenga un número asociado. Ese será su índice en la lista de posibles tokens que conforman el **vocabulario** del modelo, es decir, el conjunto de tokens posibles.

<font color='red'>TO-DO</font> ¿Cuál sería aquí el vocabulario?

### Eliminación de Stopwords

Claramente algunas palabras son más informativas que otras. Si un niño pequeño dice "gato árbol", puedes adivinar que probablemente está tratando de decir "hay un gato en el árbol". Palabras como "el" o "un" son muy comunes en español, pero normalmente no llevan mucho significado por sí solas. Estas palabras comunes se llaman **stopwords**. Eliminar stopwords puede ayudar a reducir el *ruido* en los datos y mejorar el rendimiento de los modelos de NLP.

Hay *listas predefinidas de stopwords* para explotar (¡¡descargamos una de esas listas arriba!!).

In [ ]:
stop_words = stopwords.words('english')
stop_words[:10]

Podríamos explotarlas para *filtrar* la lista anterior de `tokens`

In [ ]:
filtered_tokens = list(filter(lambda w: w.lower() not in stop_words, tokens))
filtered_tokens

<font color='red'>TO-DO</font>: ¿Qué palabras fueron eliminadas como stopwords?

In [ ]:
set(tokens) - set(filtered_tokens)

Más adelante veremos un ejemplo que muestra el efecto de eliminar stopwords.

# Extracción de Características: Bag‑of‑Words y tf-idf

En lugar de NLTK, aquí haremos uso de [scikit-learn](https://scikit-learn.org), una librería popular (casi podríamos decir *estándar*) para ML. Proporciona una serie de herramientas para procesamiento de texto, incluyendo métodos de extracción de características como Bag-of-Words y tf-idf (¡¡mantente atento!!).

Creemos algunos *documentos*

In [ ]:
docs = [
    "I love programming in Python.",
    "Python is great for NLP tasks.",
    "I enjoy learning new languages."
]

¿Cómo podemos convertir cada documento en un vector de números de *tamaño fijo*? Una vez que tenemos un diccionario, podemos simplemente contar el número de veces que cada uno de sus elementos aparece en cada *documento*. Entonces, cada documento queda representado por un vector del tamaño del vocabulario. Esta *transformación* es una tarea muy común que *scikit-learn* automatiza a través de la clase [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Lo que resulta es una representación [Bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) (o *BoW*) del corpus (en la cual cada documento se convierte en un vector del tamaño del vocabulario).

<font color='red'>TO-DO</font>: Utiliza la clase [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) (soporta *stopwords*) para *vectorizar* la `lista` de *documentos* anterior. Imprime el vector, es decir, los *conteos*, para el primer *documento*.

<font color='red'>TO-DO</font>: ¿Cuál es el vocabulario? ¿Cuántas veces aparece la 3ª palabra del vocabulario en el 2º documento?

Ahora tenemos una manera de representar cada documento de texto en un corpus como un vector de números de tamaño fijo (*conteos*). Sin embargo, incluso después de descartar las *stopwords*, que tienen poco o ningún significado, está claro que no todas las palabras son igualmente significativas. Intuitivamente, las palabras que aparecen *todo el tiempo* en un corpus (por ejemplo, la palabra "médico" en una colección de documentos sobre medicina) no son muy significativas y, viceversa, las palabras que aparecen muy poco pueden proporcionar algunas pistas útiles para una tarea. Tenemos un nombre para esta intuición: [term frequency–inverse document frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) o *tf-idf*. La idea es: para cada documento, contamos el número relativo de veces (*frecuencia*) que aparece cada palabra (*término*) y lo dividimos por el número relativo de veces que lo hace en el corpus (*frecuencia de documento*).

<font color='red'>TO-DO</font>: Utiliza la clase [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) para obtener el tf-idf en lugar de conteos *crudos*.

<font color='red'>TO-DO</font>: ¿Cuál es la palabra con los valores tf-idf no nulos más pequeños en todos los documentos?

<font color='red'>TO-DO</font>:
- ¿Cómo manejan la *dispersión* (cada documento solo incluye un pequeño número de términos del vocabulario) las clases anteriores (para evitar ocupar una cantidad enorme de memoria)?
- Después de agregar un documento **sin** la palabra "python", ¿debería aumentar o disminuir el tf-idf de esta última en los documentos originales?

## Por Qué Importan las Stopwords

Observa que, una vez que cada documento se convierte en un vector de números de tamaño fijo (el del vocabulario, digamos $N$), es fácil comparar documentos comparando sus vectores correspondientes. En principio, cualquier *distancia* en $\mathbb{R}^N$ es susceptible de ser utilizada.

Calcularemos la [similitud coseno](https://en.wikipedia.org/wiki/Cosine_similarity) entre dos oraciones cortas **con** y **sin** eliminación de stopwords para ilustrar cómo las stopwords pueden inflar las puntuaciones de similitud. Un corpus ligeramente "forzado"

In [ ]:
a_sentence = "The cat sat on the mat"
another_sentence = "A dog sat on the rug"

¿Son similares las oraciones?

**Sin** eliminar stopwords

In [ ]:
vec_all = CountVectorizer().fit_transform([a_sentence, another_sentence])
print("Cosine similarity (without stopwords):",
      cosine_similarity(vec_all)[0,1].round(3))

**Después** de eliminar las stopwords

In [ ]:
vec_ns = CountVectorizer(stop_words='english').fit_transform([a_sentence, another_sentence])
print("Cosine similarity (with stopwords):",
      cosine_similarity(vec_ns)[0,1].round(3))

Observa cómo la eliminación de stopwords reduce la similitud al excluir palabras comunes como *the* y *on*, proporcionando una sensación más verdadera de distancia semántica.

# Clasificación de Texto con Naive Bayes

Pongamos en práctica lo que hemos aprendido construyendo un *clasificador de noticias*: dado una pieza de noticia, la tarea es decidir la *categoría* (tema, tópico) entre un conjunto de opciones. Usaremos solo **2 categorías**,

In [ ]:
categories = ['rec.autos', 'rec.sport.baseball']

, del dataset `20 Newsgroups`, que se puede descargar directamente a través de *scikit-learn*.

In [ ]:
train = fetch_20newsgroups(subset='train', categories=categories, remove=('headers','footers','quotes'))
test  = fetch_20newsgroups(subset='test',  categories=categories, remove=('headers','footers','quotes'))

<font color='red'>TO-DO</font>: Echa un vistazo a un par de *posts* (ya sea del conjunto de *entrenamiento* o *prueba*), junto con su *categoría* correspondiente (es decir, etiqueta).

<font color='red'>TO-DO</font>: Convierte los documentos en *conteos* tf-idf. Para ello, entrena el *vectorizador* utilizando el conjunto de entrenamiento y, después, usa el objeto `TfidfVectorizer` resultante para transformar tanto el conjunto de entrenamiento como el de prueba por separado.

In [ ]:
# tfidf_vec = ...
# X_train = ...
# X_test = ...

<font color='red'>TO-DO</font>: Echa un vistazo al vector de números para los *posts* revisados anteriormente.

<font color='red'>TO-DO</font>: ¿Qué pasa cuando hay una palabra en el conjunto de prueba que no fue vista en el conjunto de entrenamiento (cuando haces `fit_transform` en este último)? Inventa un documento con una sola palabra no existente y observa cuántos elementos no nulos hay en su representación tf-idf (obtenida a través del `*Vectorizer` ajustado en el conjunto de entrenamiento).

Vamos a entrenar un [Naive Bayes classifier](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) sobre el conjunto de *entrenamiento*...

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, train.target)

...y utilizarlo para predecir sobre el conjunto de *test.

In [ ]:
pred = clf.predict(X_test)

<font color='red'>TO-DO</font>: ¿Cuál es la precisión general del clasificador? Necesitas comparar las predicciones anteriores contra los *targets* reales en el conjunto de *test*.

<font color='red'>TO-DO</font>: Revisa uno de los *posts* mal clasificados. ¿Eres capaz de decir a qué clase pertenece (es decir, de qué trata)?

<font color='red'>TO-DO</font>: ¿Cómo es que todos los textos terminan teniendo la misma longitud?

# Introducción al Modelado de Tópicos

El modelado de tópicos descubre tópicos latentes en un corpus *sin supervisión* (nótese que arriba teníamos *etiquetas*, es decir, supervisión, para cada noticia). Usaremos [Latent Dirichlet allocation](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) o LDA (que no debe confundirse con *Linear discriminant analysis*, también abreviado en la literatura como LDA).

Obtengamos más datos del conjunto de datos `20 Newsgroups`...pero esta vez sin hacer uso de las etiquetas (es decir, las categorías). De hecho, solo estamos usando el atributo `data` (**no** el `target`).

In [ ]:
categories = ['talk.politics.misc', 'comp.graphics', 'sci.space']
data = fetch_20newsgroups(subset='train', categories=categories,remove=('headers','footers','quotes'))
docs = data.data  # list of raw text documents

Primero *vectorizamos* el corpus usando un `CountVectorizer` básico, siendo este último más alineado con las suposiciones detrás de LDA.

In [ ]:
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(docs)

Vamos a entrenar el modelo utilizando LDA

In [ ]:
lda = LatentDirichletAllocation(n_components=3, random_state=42)
lda.fit(X)

Echemos un vistazo a las palabras más importantes (más probables) en cada tópico (es decir, `component`)

In [ ]:
vocab = vec.get_feature_names_out()
for idx, topic in enumerate(lda.components_):
    top_terms = [vocab[i] for i in topic.argsort()[-5:][::-1]]
    print(f"Topic {idx+1}: {top_terms}")

Los tópicos anteriores, ¿encajan con las *categorías* en el conjunto de datos `20 Newsgroups`?

<font color='red'>TO-DO</font>: Podría ser conveniente ignorar algunas de las palabras en los temas anteriores. Re-*ajusta* el modelo excluyendo un par de palabras: "mr" y "don".

<font color='red'>TO-DO</font>: ¿Qué pasa cuando aumentas o disminuyes el número de temas? Fíjate que, ahora mismo, tenemos más tópicos que categorías *reales* en los datos.

<font color='red'>TO-DO</font>: ¿Puede una palabra aparecer en dos tópicos diferentes? ¿Qué sucede cuando modificas el valor de `random_state` pasado a `LatentDirichletAllocation`? ¿Por qué?

# Sample questions

## What happens if a word appears in the test set but not in the training vocabulary?
- [x] It is ignored (given zero weight) by the vectorizer  
- [ ] It causes an error  
- [ ] The model guesses its meaning automatically  
- [ ] The model adds it to the vocabulary dynamically  

---

## What is the main goal of topic modeling (e.g., LDA)?
- [x] To discover hidden themes or topics in a collection of documents  
- [ ] To classify documents into predefined categories  
- [ ] To translate documents into another language  
- [ ] To remove stopwords from long texts  